In [1]:
  #!/usr/bin/env python
from __future__ import division
import argparse 
import numpy as np
import matplotlib.pyplot as plt
from fipy import *
import time
import matplotlib
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from collections import OrderedDict
import pymc3 as pm
import theano.tensor as tt
import arviz as az
import pickle

In [2]:
# from IPython.display import Image
# from IPython.core.display import HTML 
# Image(filename = "EIT.png") # width=700, height=700

In [3]:
class EIT_fipy():
    """
    Class that solves forward electrostatic EIT BVP for different boundary conditions using fipy package for a circular damage.
    x_center:  Location of the damage center.
    """
    def __init__(self, nx1, nx2, x1_center, x2_center):
        
        # defect with radius r at x_center
        self.r = 0.1
        self.x_center = np.array([[x1_center, x2_center]])
        
        # electrical conductivities
        self.a_d = 1.5
        self.a_o = 10
        
        self.Expts_description = {
                                'Expt-1': 'Unit current applied on all the sides',
                                'Expt-2': 'Unit current on left and right, no current on the other sides',
                                'Expt-3': 'Unit current on top and bottom, no current on the other sides'
                                }
        # Applying unit current on the Neumann boundaries
        self.j = 1 # unit current
        # Boundary conditions value: [left, right, top, bottom]
        self.BCs = {
                'Expt-1': [self.j/self.a_o, self.j/self.a_o, self.j/self.a_o, self.j/self.a_o],
                'Expt-2': [self.j/self.a_o, self.j/self.a_o, 0, 0],
                'Expt-3': [0, 0, self.j/self.a_o, self.j/self.a_o]
                }

        # defining mesh to get cellcenters
        self.Lx1 = 1.  # always put . after 1 
        self.Lx2 = 1.  # always put . after 1 
        self.nx1, self.nx2 = nx1, nx2 #100, 100 #64, 64
        self.f = 0. # source
        
        # define mesh
        self.mesh = Grid2D(nx=self.nx1, ny=self.nx2, dx=self.Lx1/self.nx1, dy=self.Lx2/self.nx2) # with nx1*nx2 number of cells/cellcenters/pixels/pixelcenters
        self.x_fipy = self.mesh.cellCenters.value.T ## fipy solution (nx1*nx2,2) matrix # same as cellcenters defined above
        
    def electrical_conductivity(self):
        """
        Getting required electrical conductivities throughout the domain.
        """
        d = np.linalg.norm(self.x_fipy-self.x_center, axis=1)
        a = np.zeros(len(d))
        filt = np.less_equal( d**2, self.r**2 )
        a[filt] = self.a_d # when True
        a[~filt] = self.a_o # when False
        return a
         
    def plot_electrical_conductivity(self, save_fig='False'):
        """
        Plotting the electrical conductivity.
        """
        x1_f = self.x_fipy[:,0][:,None] # x1-coordinates of cell centers (nx1*nx2,1) matrix
        x2_f = self.x_fipy[:,1][:,None] # x2-coordinates of cell centers (nx1*nx2,1) matrix
        a = self.electrical_conductivity()

        fig = plt.figure()
        axes = fig.gca()
        col = axes.contourf(x1_f.reshape((self.nx1, self.nx2)), x2_f.reshape((self.nx1, self.nx2)), a.reshape((self.nx1, self.nx2)), 20, vmin=1.5, vmax=10.0, cmap='Oranges')
        plt.colorbar(col) 
        plt.xlabel('$x1$', fontsize=12)
        plt.ylabel('$x2$', fontsize=12)
        
        if save_fig == 'True':
            plt.savefig('conductivity.png') 
        plt.show()
            
    def solve(self, expt = 'Expt-1'):
        """
        FIPY solution of the specified experiment.
        """
        # define cell and face variables
        phi = CellVariable(name='$T(x)$', mesh=self.mesh, value=0.)
        A = CellVariable(name='$D(x)$', mesh=self.mesh, value=1.0) ## coefficient in diffusion equation
        # A = FaceVariable(name='$D(x)$', mesh=self.mesh, value=1.0) ## coefficient in diffusion equation
        source = CellVariable(name='$f(x)$', mesh=self.mesh, value=1.0)

        # apply boundary conditions
        # homogeneous Neumann
        phi.faceGrad.constrain(self.BCs[expt][0], self.mesh.facesLeft)
        phi.faceGrad.constrain(self.BCs[expt][1], self.mesh.facesRight)
        phi.faceGrad.constrain(self.BCs[expt][2], self.mesh.facesTop)
        phi.faceGrad.constrain(self.BCs[expt][3], self.mesh.facesBottom)

        # setup the diffusion problem
        eq = -DiffusionTerm(coeff=A) == source

        source.setValue(self.f)
        a = self.electrical_conductivity()
        A.setValue(a)

        eq.solve(var=phi)

        u_fipy = phi.value[:][:, None] ## fipy solution  (nx1*nx2,1) matrix
        return u_fipy
            
    def forward_process(self, expt = 'Expt-1'):
        
        x1_f = self.x_fipy[:,0][:,None] # x1-coordinates of cell centers (nx1*nx2,1) matrix
        x2_f = self.x_fipy[:,1][:,None] # x2-coordinates of cell centers (nx1*nx2,1) matrix
        u_fipy = self.solve(expt)
        data = {
                'x1_f': x1_f.flatten(),
                'x2_f': x2_f.flatten(),
                'u_fipy': u_fipy.flatten()
                }
        df = pd.DataFrame(data)
        
        N = 50 # Number of observations on each boundary
        
        p1_ = np.asarray( list(OrderedDict.fromkeys(x1_f.flatten())) ).reshape(-1,1)
        p2_ = np.asarray( list(OrderedDict.fromkeys(x2_f.flatten())) ).reshape(-1,1)
        
        p1 = p1_[np.linspace(1,self.nx1-2,N, dtype = int), :].flatten()
        p2 = p2_[np.linspace(1,self.nx2-2,N, dtype = int), :].flatten()
        
        cond_1 = (df['x1_f'].isin(p1)) & (df['x2_f'] == p2_[0][0]) # bottom
        cond_2 = (df['x1_f'].isin(p1)) & (df['x2_f'] == p2_[-1][0]) # top
        cond_3 = (df['x1_f'] == p1_[0][0]) & (df['x2_f'].isin(p2)) # left
        cond_4 = (df['x1_f'] == p1_[-1][0]) & (df['x2_f'].isin(p2)) # right

        idx =  cond_1 | cond_2 | cond_3 | cond_4
        df_needed = df[idx].reset_index(drop=True)
        
        return df_needed['u_fipy'].tolist()

        
    def solve_all(self):
        """
        FIPY solution of all the experiments.
        """
        u_fipy_dict = {}
        for expt, _ in self.BCs.items():
            u_fipy_dict[expt] = self.solve(expt)
        return u_fipy_dict
            
    def plot_all(self, u_fipy_dict, save_fig = 'False'): 
        """
        Plotting FIPY solution of all the experiments.
        """            
        fig = plt.figure(figsize=(8*len(self.BCs),6))
        cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["navy","blue","deepskyblue","limegreen","yellow","darkorange","red","maroon"])
        
        for i, expt in zip(range(len(self.BCs)), self.BCs.keys()):
            x1_f = self.x_fipy[:,0][:,None] # x1-coordinates of cell centers (nx1*nx2,1) matrix
            x2_f = self.x_fipy[:,1][:,None] # x2-coordinates of cell centers (nx1*nx2,1) matrix
            
            ax = fig.add_subplot(1, len(self.BCs), i+1)
            col = ax.contourf( x1_f.reshape((self.nx1, self.nx2)), x2_f.reshape((self.nx1, self.nx2)), u_fipy_dict[expt].reshape((self.nx1, self.nx2)), 100, cmap='twilight') # set levels as previous levels
            # This is the fix for the white lines between contour levels (https://stackoverflow.com/questions/8263769/hide-contour-linestroke-on-pyplot-contourf-to-get-only-fills)
            for j in col.collections:
                j.set_edgecolor("face")
            plt.colorbar(col)
            plt.title(f'{expt} (FVM solution)',fontsize=14)
            plt.xlabel('$x1$', fontsize=12)
            plt.ylabel('$x2$', fontsize=12)
            
        if save_fig == 'True':
            plt.tight_layout()
            plt.savefig('FVM_solution.png') 


In [4]:
x1_center_gt_list = np.arange(0.2,0.8,0.01)
x2_center_gt_list = np.arange(0.2,0.8,0.01)
x1v, x2v = np.meshgrid(x1_center_gt_list, x2_center_gt_list, indexing='xy')
x_center_gt_set = np.hstack((x1v.reshape(-1,1), x2v.reshape(-1,1)))
print(x_center_gt_set.shape)
print(x_center_gt_set)

(3721, 2)
[[0.2  0.2 ]
 [0.21 0.2 ]
 [0.22 0.2 ]
 ...
 [0.78 0.8 ]
 [0.79 0.8 ]
 [0.8  0.8 ]]


In [5]:
def data_generation(nx1, nx2, x_center_gt_set, expt):
    #define matrices to save results 
    inputs = x_center_gt_set
    outputs = np.zeros((x_center_gt_set.shape[0], 200))

    for i in range(x_center_gt_set.shape[0]):
        obj_gt = EIT_fipy(nx1, nx2, x_center_gt_set[i,0], x_center_gt_set[i,1])
        u_fipy = obj_gt.solve(expt = expt) 
        #save data 
        outputs[i] = obj_gt.forward_process(expt = expt) # 200 measurements on all the 4 boundaries
    
    cellcenters = obj_gt.x_fipy 
    
    return cellcenters, inputs, outputs

In [6]:
data_dict = {}
expts_list = ['Expt-1', 'Expt-2', 'Expt-3']
nx1, nx2 = 395, 395
save = True
for expt in expts_list: 
    cellcenters, inputs, outputs = data_generation(nx1, nx2, x_center_gt_set, expt)
    data_dict[expt+'_inputs'] = inputs
    data_dict[expt+'_outputs'] = outputs
    if save==True:
        np.save(expt+'_inputs_'+'cellcenters_nx1='+str(nx1)+'_nx2='+str(nx2)+'.npy',
                data_dict[expt+'_inputs'])
        np.save(expt+'_outputs_'+'cellcenters_nx1='+str(nx1)+'_nx2='+str(nx2)+'.npy', 
                data_dict[expt+'_outputs'])
        np.save('cellcenters_nx1='+str(nx1)+'_nx2='+str(nx2)+'.npy', cellcenters)

In [7]:
#checks 
cellcenters.shape, data_dict['Expt-3'+'_inputs'].shape, data_dict['Expt-3'+'_outputs'].shape

((156025, 2), (3721, 2), (3721, 200))

# checks

In [8]:
print(cellcenters)

[[0.00126582 0.00126582]
 [0.00379747 0.00126582]
 [0.00632911 0.00126582]
 ...
 [0.99367089 0.99873418]
 [0.99620253 0.99873418]
 [0.99873418 0.99873418]]


In [9]:
import torch

In [10]:
loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.from_numpy(data_dict[expt+'_inputs']),torch.from_numpy(data_dict[expt+'_outputs'])), batch_size=torch.from_numpy(data_dict[expt+'_inputs']).shape[0], shuffle=True)
loader
for x, y in loader:
    print(x.shape, y.shape)

torch.Size([3721, 2]) torch.Size([3721, 200])
